In [1]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report, accuracy_score, confusion_matrix, roc_auc_score, precision_recall_curve
from imblearn.over_sampling import SMOTE
from imblearn.pipeline import Pipeline as ImbPipeline

In [2]:
# Load your dataset
df = pd.read_csv('filtered_student_data.csv', low_memory = False)

In [3]:
# Separate features and target variable
X = df.drop('dropout', axis=1)
y = df['dropout']

# Identify categorical and numerical columns
categorical_cols = [col for col in X.columns if X[col].dtype == 'object']
numerical_cols = [col for col in X.columns if X[col].dtype in ['int64', 'float64']]

# Preprocessing for numerical data
numerical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())])

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)])

In [4]:
# Define the model with class weights
model = RandomForestClassifier(n_estimators=100, random_state=0, class_weight='balanced')

# Create a pipeline
pipeline = ImbPipeline(steps=[('preprocessor', preprocessor),
                              ('smote', SMOTE(random_state=0)),
                              ('model', model)])

# Split data into train and test sets
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=0)

In [ ]:
# Train the model
pipeline.fit(X_train, y_train)

# Make predictions
y_pred = pipeline.predict(X_test)

# Evaluate the model
print("Accuracy:", accuracy_score(y_test, y_pred))
print("Classification Report:\n", classification_report(y_test, y_pred))

# Confusion Matrix
cm = confusion_matrix(y_test, y_pred)
print("Confusion Matrix:\n", cm)

# ROC AUC Score
roc_auc = roc_auc_score(y_test, y_pred)
print("ROC AUC Score:", roc_auc)

# Precision-Recall Curve
precision, recall, thresholds = precision_recall_curve(y_test, y_pred)
print("Precision-Recall Curve:\n", list(zip(precision, recall, thresholds)))

In [11]:
print(categorical_cols)
print(numerical_cols)

['birth_certificate_id', 'sex', 'father_educational_attainment', 'mother_educational_attainment', 'grade_id', 'relation_with_guardian', 'pwd_type', 'pwd_degree', 'marital_status']
['is_orphan', 'is_never_been_to_school', 'is_ethnic', 'parents_income', 'previous_dropout', 'received_any_treatment', 'newly_admitted']
